![](pictures/shopping_propensity.png)

<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
Ноутбук от 30.09.23 
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>   
**_"Data Science Analyst"_  
_МОДЕЛЬ СКЛОННОСТИ КЛИЕНТОВ К ПОКУПКАМ В ФЕДЕРАЛЬНОЙ СЕТИ ПРОДАЖ СПОРТИВНЫХ ТОВАРОВ_**  

Выполнено по технологии CRISP-DM:  
Аргентов Сергей, г.Липецк  
https://argentov.pro
<b>  </b>  
<b>  </b>  

<h1>СОДЕРЖАНИЕ<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Предобработка-данных" data-toc-modified-id="1.-Предобработка-данных-1">1. Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#1.1.-Анализ-источников-данных" data-toc-modified-id="1.1.-Анализ-источников-данных-1.1">1.1. Анализ источников данных</a></span></li><li><span><a href="#1.2.-Устранение-пропусков-в-данных" data-toc-modified-id="1.2.-Устранение-пропусков-в-данных-1.2">1.2. Устранение пропусков в данных</a></span><ul class="toc-item"><li><span><a href="#1.2.1.-Проверим-гипотезу-что-для-товаров-с-пропуском-в-признаке-&quot;гендерная-принадлежность-потенциальных-покупателей-товара&quot;-нет-признаков-принадлежности-к-полу-в-наименовании-товара" data-toc-modified-id="1.2.1.-Проверим-гипотезу-что-для-товаров-с-пропуском-в-признаке-&quot;гендерная-принадлежность-потенциальных-покупателей-товара&quot;-нет-признаков-принадлежности-к-полу-в-наименовании-товара-1.2.1">1.2.1. Проверим гипотезу что для товаров с пропуском в признаке "гендерная принадлежность потенциальных покупателей товара" нет признаков принадлежности к полу в наименовании товара</a></span></li><li><span><a href="#1.2.2.-Проверим-гипотезы-о-причинах-пропусков-знаений-в-признаке-&quot;цвет&quot;" data-toc-modified-id="1.2.2.-Проверим-гипотезы-о-причинах-пропусков-знаений-в-признаке-&quot;цвет&quot;-1.2.2">1.2.2. Проверим гипотезы о причинах пропусков знаений в признаке "цвет"</a></span><ul class="toc-item"><li><span><a href="#1.2.2.1.-Проверим-попала-ли-информация-о-&quot;цвете&quot;-в-наименование-товара" data-toc-modified-id="1.2.2.1.-Проверим-попала-ли-информация-о-&quot;цвете&quot;-в-наименование-товара-1.2.2.1">1.2.2.1. Проверим попала ли информация о "цвете" в наименование товара</a></span></li><li><span><a href="#1.2.2.2.-Проверим-есть-ли-одноимённые-товары-с-наличием-значения-в-признаке-&quot;цвет&quot;-для-товаров-с-пропущенным-значением-в-признаке-&quot;цвет&quot;" data-toc-modified-id="1.2.2.2.-Проверим-есть-ли-одноимённые-товары-с-наличием-значения-в-признаке-&quot;цвет&quot;-для-товаров-с-пропущенным-значением-в-признаке-&quot;цвет&quot;-1.2.2.2">1.2.2.2. Проверим есть ли одноимённые товары с наличием значения в признаке "цвет" для товаров с пропущенным значением в признаке "цвет"</a></span></li></ul></li></ul></li></ul></li></ul></div>

In [1]:
# Загрузка биьлиотек
import pandas as pd
import sqlite3 as db

import datetime as dt
import multiprocessing
from multiprocessing import Pool, Manager

# 1. Предобработка данных

## 1.1. Анализ источников данных

In [2]:
# Предоставленная база данных
with db.connect('data/result_data/shop_database.db') as con:
    rows = con.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    dfs = {}
    for row in rows:
        dfs[row[0]] = pd.read_sql(f'SELECT * FROM {row[0]}', con)
        print(f'\n{row[0]}')
        print(dfs[row[0]].info())


purchases
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786260 entries, 0 to 786259
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           786260 non-null  int64  
 1   product      786260 non-null  object 
 2   colour       666736 non-null  object 
 3   cost         786260 non-null  int64  
 4   product_sex  471548 non-null  float64
 5   base_sale    786260 non-null  int64  
 6   dt           786260 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 42.0+ MB
None

personal_data_coeffs
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104989 entries, 0 to 104988
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             104989 non-null  int64  
 1   lbt_coef       104989 non-null  float64
 2   ac_coef        104989 non-null  float64
 3   sm_coef        104989 non-null  float64
 4   personal_coef  10

## 1.2. Устранение пропусков в данных

Пропущенные значения есть в таблице `product_sex` (15%) и `color` (40%). Это может быть следствием технической/организационой ошибок при заполнении данных или может отображать следующее:  
- для признака `product_sex` в случае унисекс товаров данные о гендерной принадлежности потенциальных покупателей товара не проставлялась,
- для признака `colour` часть информации могла попасть в наименование товара, часть информации может быть получены из информации по одноимённым товарам, а в случае когда товар не имеет цветовой характеристики (пищевые продукты, лекарственные средствва и т.п. или товары с незафиксированным признаком цвета по техническим или организационным причинам) цвет товара не проставлялся.  
Проверим вышеуказанные гипотизы

### 1.2.1. Проверим гипотезу что для товаров с пропуском в признаке "гендерная принадлежность потенциальных покупателей товара" нет признаков принадлежности к полу в наименовании товара

In [3]:
# Соберём уникальный перечень покупок
df_product_sex = dfs['purchases'].groupby('product').agg({'product_sex': 'max'})
df_product_sex['product'] = df_product_sex.index
df_product_sex.reset_index(drop=True, inplace=True)
print(f'Уникальные значения гендерной принадлежности '
      f'потенциального покупателя: {df_product_sex.product_sex.unique()}\n'
      f'Количесвто уникальных товаров: {len(df_product_sex)}')

Уникальные значения гендерной принадлежности потенциального покупателя: [nan  0.  1.]
Количесвто уникальных товаров: 23145


In [4]:
# Проверим какие покупки и с какими признаками гендерной принадлежности
# имеют в наименовании товара слова 'муж, жен'
def check_product_name_sex(x, fit) -> int:
    if fit in x:
        return 1
    else:
        return 0

df_product_sex['муж'] = df_product_sex['product'].\
                        apply(lambda x:check_product_name_sex(x,'муж'))
df_product_sex['жен'] = df_product_sex['product'].\
                        apply(lambda x: check_product_name_sex(x,'жен'))

In [5]:
# Проверим все ли товары,
# не имеющие гендерной принадлежности потенциального покупателя
# не имеют в названии явного указания на пол 'муж', 'жен'
df_product_sex['унисекс'] = df_product_sex.\
                            apply(lambda x: 1
                                 if x['муж'] == 0 and x['жен'] == 0
                                 else 0,
                                 axis=1)
# Определим количество стрк где в названиии продукта нет гендерной принадлежности
# потенциального покупатнлы и не проставлены соответсвующие признаки
len(df_product_sex[(df_product_sex['унисекс'] != 1) & (df_product_sex.product_sex.isna())])

0

**Вывод:** В признаке `product_sex` пропущенные данные с большой вероятностью обозначают что товар пренадлежит к категории `унисекс`. Для возможности использования данного признака в дальнейшем проставим ему в датасете значение 2.0

In [6]:
# Заменим пустые значения признака `product_sex` на значение 2.0
dfs['purchases'].product_sex = dfs['purchases'].product_sex.fillna(2.0)

### 1.2.2. Проверим гипотезы о причинах пропусков знаений в признаке "цвет"

In [7]:
# первоначальный процент строк без заполненного значения признака "цвет"
round((len(dfs['purchases'][dfs['purchases']['colour'].isna()])/len(dfs['purchases'])) * 100, 2)

15.2

#### 1.2.2.1. Проверим попала ли информация о "цвете" в наименование товара

In [8]:
# Создалим множество используемых цветов в товарах
colors = list(dfs['purchases'][~dfs['purchases']['colour'].isna()]['colour'].
              apply(lambda x: x.split('/')))
colors_all = []
for color in colors:
    if isinstance(color, list):
        colors_all.extend(color)
colors_set = set(colors_all)
print(f'Количесвто уникальных цветов: {len(colors_set)}')

Количесвто уникальных цветов: 361


In [9]:
# Функция проверки наличия информации о цвете товара в наименовании товара
def check_name_product(product):
    colors = []
    for word in product.split(' '):
        if word in colors_set:
            colors.append(word)
    if len(colors):
        return '/'.join(colors)

In [10]:
# Получим датасет с отсутствующими значениями в признаке "цвет"
df_non_colors = dfs['purchases'][dfs['purchases']['colour'].isna()]
df_non_colors['colorus_new'] = df_non_colors['product'].apply(lambda x: check_name_product(x))
indexs_color = df_non_colors[~df_non_colors['colorus_new'].isna()]['colorus_new']

/var/folders/5h/1vkpjwr90l19cmzfgsfrcpvm0000gn/T/ipykernel_5496/1099927576.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_colors['colorus_new'] = df_non_colors['product'].apply(lambda x: check_name_product(x))


In [11]:
# Добавим полученные цвета в столбец датасета с покупками
for index, new_color in zip(indexs_color.index, indexs_color[indexs_color.index]):
    dfs['purchases'].loc[index, ['colour']] = new_color

In [12]:
# Процент строк без заполненного значения признака "цвет" после первого этапа предобработки данных
round((len(dfs['purchases'][dfs['purchases']['colour'].isna()])/len(dfs['purchases'])) * 100, 2)

15.04

**Вывод:** Данным методом удалось добавить только 0.16% значений признака "цвет"

#### 1.2.2.2. Проверим есть ли одноимённые товары с наличием значения в признаке "цвет" для товаров с пропущенным значением в признаке "цвет"

Проставим каждому товару из списка товаров без признака "цвет" наиболее часто встречающееся значение цвета.  
Для экономии времени при расчёте выполним присвоение в мультипроцессорном режиме

In [13]:
# Загрузим функции поиска во всех записях о товаре наиболее часто встречающихся значений признака "цвет"
# для наименований товаров, по которым есть записи с незаполненым признаком "цвет"
# (функции написаны в отдельном модуле для обеспечения требований блиотеки  multiprocessing)
from package.preparation import add_freq_product_color, freq_product_color

In [14]:
# Сформируем датасеты:

# df_1 - датасет в котором среди уникальных наименований товара будем искать такие товары,
#        в которых проставлено наиболее часто встречающееся значение признака "цвет" для этого товара 
df_1 = dfs['purchases'][['product', 'colour']]

# df_2 - датасет в котором сформируем список всех уникальных товаров с пропущенным значение признака "цвет"
df_2 = df_1[df_1['colour'].isna()].groupby('product').agg({'colour': 'max'})
df_2.reset_index(inplace=True)

In [15]:
######   ЗАПУСКАЕМ РАСЧЁТЫ В МНОГОПРОЦЕССОРНОМ РЕЖИМЕ   ######

In [16]:
# 1. Рассчитываем количество ядер процесса, которые можно задействовать для вычислений
#    (уменьшаем количество доступных ядер на 1 ядро, которое оставляем на системные потребности)
n_cpu = multiprocessing.cpu_count() - 1
print('Количество доступных процессоров:', n_cpu)

# 2. Расчитаем минимальный размер обрабатываемых парралельно подмассивов
# (по кратности ядер, которые будем использовать)
val_sub = int(len(df_2)/n_cpu)
print('Размеры подмассивов:', val_sub, '\n')

Количество доступных процессоров: 7
Размеры подмассивов: 1682 



In [17]:
# Ииспользуем признак работы в главном модуле (для обеспечения требований блиотеки  multiprocessing)
if __name__ == '__main__':
    
    # Начнём замер времени
    start_time = dt.datetime.now()
    print('ВРЕМЯ СТАРТА РАСЧЁТА:', start_time, '\n')
    
    # Распарралелим расчёты между доступными ядрами процессора
    # 3. Создаём менеджера парралельных процессов
    #    и его пространство имён для обмена данными между процессами
    with Manager() as mng:
        ns = mng.Namespace()
        ns.dfs = mng.list()
        df = df_1
        df_u = df_2
        print(f'Размеры массивов: {len(df_u)} и {len(df)}\n\n')
        
        # 4. Делим датасет на составляющие,
        #    которые будут рассчитываться в парралельных процессах
        for i in range(n_cpu - 1):
            df = df_1.copy()
            ns.dfs.append((df_u[val_sub*i:val_sub*(i+1)], df.copy()))
            print(f'Подготовлен {i+1} массив размером: {len(ns.dfs[i][0])} для сравнения с {len(ns.dfs[i][1])}')
        ns.dfs.append((df_u[(n_cpu-1)*val_sub:], df.copy()))
        print(f'Подготовлен последний массив размером: {len(ns.dfs[i+1][0])} для сравнения с {len(ns.dfs[i+1][1])}')

        # 5. Запустим пул парралельных процессов вычислений
        #    и результат объединим в единый датафрейм
        with Pool(processes=n_cpu) as p:
            print(f'... запущен параллельный расчёт на {n_cpu} ядрах процессора ...')
            df_non_colours = pd.concat(list(p.map(add_freq_product_color, ns.dfs)), axis=0)

    # Закончим замер времени
    finish_time = dt.datetime.now()
    print('\n\nВРЕМЯ ОКОНЧАНИЯ РАСЧЁТА:', finish_time)
    print('ВРЕМЯ ВЫПОЛНЕНИЯ:', finish_time - start_time, '\n' * 2)

ВРЕМЯ СТАРТА РАСЧЁТА: 2023-10-02 02:43:21.681496 

Размеры массивов: 11775 и 786260


Подготовлен 1 массив размером: 1682 для сравнения с 786260
Подготовлен 2 массив размером: 1682 для сравнения с 786260
Подготовлен 3 массив размером: 1682 для сравнения с 786260
Подготовлен 4 массив размером: 1682 для сравнения с 786260
Подготовлен 5 массив размером: 1682 для сравнения с 786260
Подготовлен 6 массив размером: 1682 для сравнения с 786260
Подготовлен последний массив размером: 1683 для сравнения с 786260
... запущен параллельный расчёт на 7 ядрах процессора ...


ВРЕМЯ ОКОНЧАНИЯ РАСЧЁТА: 2023-10-02 02:49:37.124587
ВРЕМЯ ВЫПОЛНЕНИЯ: 0:06:15.443091 




In [18]:
# Сформируем датасет с вновь заполненным признаком "цвет"
df_3 = df_non_colours[~df_non_colours['freq_color'].isna()]

# Создадим словарь восстановленных данных о признаке "цвет"
recovered_color_data = {}
for key, value in zip(list(df_3['product']), list(df_3['freq_color'])):
    recovered_color_data[key] = value

In [19]:
# Функция добавления восстановленных данных в признак "цвет"
def replace_none(product, colour, recovered_color_data=recovered_color_data):
    if pd.isna(colour):
        if product in list(recovered_color_data.keys()):
            return recovered_color_data[product]
        else:
            return None
    else:
        return colour

In [20]:
# Добавим восстановленные записи признака "цвет" в исходный датасет с товарами
dfs['purchases']['colour'] = dfs['purchases'].apply(lambda x: replace_none(x['product'], x['colour']), axis=1)

In [21]:
# Процент строк без заполненного значения признака "цвет" после первого этапа предобработки данных
round((len(dfs['purchases'][dfs['purchases']['colour'].isna()])/len(dfs['purchases'])) * 100, 2)

0.42

**Вывод:** Данным методом удалось добавить только 14.62% значений признака "цвет" и довести уровень заполненности данного признака до 99,6%

In [ ]:
# Посчитаем сколько % записей с признаком "цвет" удалось восстановить
round((len(df_non_colours[~df_non_colours['freq_color'].isna()]) / len(dfs['purchases'])) * 100, 2)

In [ ]:
# добавим восстановленные записи в датасет с товарами
# for ind, prod, colour in zip(dfs['purchases'].index, dfs['purchases'].index, dfs['purchases'].colour):
#     if pd.isna(colour):
#         if prod in recovered_color_data.keys():
#             print(recovered_color_data[prod])